## Text Network

### Install dependencies

In [2]:
import re
import itertools
import spacy
from nltk import FreqDist, bigrams
import numpy as np
import pandas as pd
import networkx as nx
from pyvis.network import Network
import seaborn as sns
from collections import Counter
sns.set(rc={'figure.figsize':(15,7)})

#nltk.download('stopwords')
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from youtube_transcript_api import YouTubeTranscriptApi

In [3]:
def generate_transcript(id):
	transcript = YouTubeTranscriptApi.get_transcript(id,languages=['en'])
	script = ""

	for text in transcript:
		t = text["text"]
		if t != '[Music]':
			script += t + " "
		
	return script, len(script.split())

id = '-nZkP2b-4vo'

#ID é a parte final do link, depois do "=", exemplo 
#https://www.youtube.com/watch?v=L_jWHffIx5E
#L_jWHffIx5E

transcript, no_of_words = generate_transcript(id)
print(transcript,no_of_words)

How did humanity come to accept rectangular pieces of pulped trees as something to spend eight to ten hours a day working for? It's a pretty insane story. This change from hard currency like gold or silver is a really huge deal. Without it, we couldn't possibly have the massive industrial and post-industrial economies we know today. This change revolutionized how we do business and forever altered how governments were financed. Learning about this massive sea change in how we as a species thought about money can help us reflect on our current historical shift from seeing paper as money, to seeing bits, seeing digital ones and zeros as money. But before we can get the exciting story of people trying to convince other people that paper was worth something, to understand why this is such a huge deal, we have to discuss a bit about how we thought about money before paper. If we go way back to the beginning of society, we find trade. Before early humans even really settled down, there's evi

In [4]:
transcript = YouTubeTranscriptApi.get_transcript(id,languages=['en'])
for text in transcript:
    print(text['start'])
    print(text['text'])

0.0
How did humanity come to accept rectangular pieces of pulped trees as something to spend eight to ten hours a day working for?
13.0
It's a pretty insane story.
15.16
This change from hard currency like gold or silver
18.52
is a really huge deal.
19.96
Without it, we couldn't possibly have the massive industrial
22.78
and post-industrial economies we know today.
25.28
This change revolutionized how we do business
27.52
and forever altered how governments were financed.
30.38
Learning about this massive sea change
32.42
in how we as a species thought about money
35.06
can help us reflect on our current historical shift
37.56
from seeing paper as money, to seeing bits,
40.36
seeing digital ones and zeros as money.
42.4
But before we can get the exciting story of people
44.68
trying to convince other people that paper was worth something,
47.26
to understand why this is such a huge deal,
49.62
we have to discuss a bit about how we thought about money before paper.
53.46
If we go way ba

In [14]:
timeDict = {}

def getTimes(txt,e,timeDict):
	i = 0
	while i < len(transcript):
		if re.search(txt,transcript[i]['text']):
			if timeDict[txt] == []:
				timeDict[txt].append(transcript[i]['start'])
			elif timeDict[txt][len(timeDict[txt])-1] + e < transcript[i]['start']:
				timeDict[txt].append(transcript[i]['start'])
		i += 1

{'money': [32.42, 173.849, 213.269, 254.31, 296.89, 344.74, 470.26]}


### Auxiliary functions

In [5]:
toktok = ToktokTokenizer()
stpwrd = stopwords.words('english')
stop_list = (['uh', 'also', 'oh', 'um', 'yeah', 'use', 'lot', 'put', 'get', 'would', 'gonna', 'really', 'much', 'actually'])
stpwrd.extend(stop_list)


'''word_tokens = toktok.tokenize(example_sent)

filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

filtered_sentence = []

for w in word_tokens:
	if w not in stop_words:
		filtered_sentence.append(w)

print(word_tokens)
print(filtered_sentence)'''


def filtered_sentence(example_sent):
    word_tokens = toktok.tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stpwrd]
    return " ".join(filtered_sentence)

def remove_stopwords(text):
    doc = filtered_sentence(text)
    print(" ".join([token.text for token in doc if token.is_stop == False]))
    return " ".join([token.text for token in doc if token.is_stop == False])

def pre_processing(text):
    text = filtered_sentence(text)
    text = re.sub(r'[^\w\s]', '', text) #remove punctuation
    text = re.sub(r"\s{2,}", " ", text)
    return text

def co_occurrence(text):
    corpus = list(itertools.chain.from_iterable([text.split(' ')]))
    vocab = list(set(corpus))
    vocab_index = {word: i for i, word in enumerate(vocab)}
    bi_grams = list(bigrams(corpus))
    bigram_freq = FreqDist(bi_grams).most_common(len(bi_grams)) 
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab))) 
    
    #bigram = ((word n, word n+1), num_occurrences)
    for bigram in bigram_freq:
        current = bigram[0][1] #word n+1
        previous = bigram[0][0] #word n
        count = bigram[1] #num_occurrences
        pos_current = vocab_index[current] #obtain id for word n+1 
        pos_previous = vocab_index[previous] #obtain id for word n 
        co_occurrence_matrix[pos_current][pos_previous] = count
        
    return co_occurrence_matrix, vocab_index

def structure_text_network(matrix, vocab_index):
    data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
    data_stack = data_matrix.stack()
    structure = data_stack[data_stack >= 1].rename_axis(('source', 'target')).reset_index(name='weight')
    return structure[(structure.source != " ") & (structure.target != " ")]

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\emanu/nltk_data'
    - 'c:\\Program Files\\Python310\\nltk_data'
    - 'c:\\Program Files\\Python310\\share\\nltk_data'
    - 'c:\\Program Files\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\emanu\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


### Generate text network

In [ ]:
from spacy.lang.pt.stop_words import STOP_WORDS
nlp = spacy.load("pt_core_news_sm")
#STOP_WORDS.add('gotta')

In [ ]:
text = transcript

In [ ]:
text_cleaned = pre_processing(text)
matrix, vocab = co_occurrence(text_cleaned)
structure_network = structure_text_network(matrix, vocab)

In [ ]:
structure_network

,source,target,weight
0,transactions,economic,1.0
1,transactions,largestscale,1.0
2,early,trade,1.0
3,huge,understand,1.0
4,huge,silver,1.0
...,...,...,...
735,sector,financial,1.0
736,possibly,Without,1.0
737,thus,value,1.0
738,heavy,money,1.0


In [ ]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])

### PageRank

In [ ]:
text_network = nx.DiGraph()
text_network = nx.from_pandas_edgelist(structure_network, 'target', 'source', ['weight'])

In [ ]:
def pagerank_centrality(G):
    centrality =  nx.pagerank(G)
    return sorted(centrality.items(), key=lambda x: x[1], reverse=False)

In [ ]:
centrality = pagerank_centrality(text_network)

#dicionário da centralidade
cen_dict = {}
max_cen = 0
for x in centrality:
    cen_dict[x[0]] = x[1]
    if max_cen < x[1]:
        max_cen = x[1]

### Graph Visualization

In [ ]:
# lista dos nós
name_lst = [x[0] for x in centrality]

# lista de nós a serem removidos, estamos mantendo os 25 maiores
rmv_lst = name_lst[:-25]

# exibição do nome dos nós para a visualização
for i in text_network.nodes:
    text_network.nodes[i]['label'] = i

# redução do grafo mantendo a conectividade
for i in rmv_lst:
    maxi = 0
    for j in text_network.neighbors(i):
        w = name_lst.index(j)
        if maxi < w:
            maxi = w
            merge_node = j
            
    for j in text_network.neighbors(i):
        if not (j == merge_node or (j in text_network.neighbors(merge_node))):
            text_network.add_edge(merge_node, j, weight=text_network[i][j]['weight']/2)
    text_network.remove_node(i)

In [ ]:
nt = Network('800px', '1000px', notebook=True)
nt.from_nx(text_network)
for edg in nt.edges:
    # espessura e física das arestas
    #edg['width'] = text_network[i][j]['weight']**4
    #if edg['weight'] < 1:
    edg['physics'] = False

for n in nt.nodes:
    # tamanho dos nós
    n['shape'] = 'ellipse'
    size = 30
    n['font'] = str(int(size))+'px arial white'
    n['color'] = '#7f333f'
    n['labelHighlightBold'] = True
    
    
for n in nt.nodes:
    # links nos nós
    n['title'] = "<a href='http://www.google.com\'>google <br> <a href='http://www.google.com\'>yout"
    #print(n['size'])

    
nt.show("text_network.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [ ]:
#print(nx.adjacency_matrix(text_network))
#print(nt.nodes)
for node in nt.nodes:
    print(node['label'])

trade
think
want
run
something
often
started
one
problem
time
money
used
gold
enough
even
value
good
somebody
paper
commodity
massive
currency
like
worth
people


In [ ]:
um, yeah, use, lot, put, get, would, gonna, really, much, actually